# Collecting Data
I will use the yahoo finance API to get the start and end price for our benchmark, DIA. DIA is the oldest Dow Jones mutual fund. Yahoo finance has data going back to 1998 for this fund. We will get the prices for the components of the Dow Jones circa 1997 (as determined from [this wikipedia article](https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average)) for the same time period as our benchmark.

I will keep transformation and cleaning to a minimum during the collection phase.

In [256]:
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np
import json
import os
from time import sleep
pd.set_option('display.max_rows', 10)

Setup directories if necessary

In [2]:
try:
    os.makedirs("data/raw") 
except FileExistsError as e:
    print(e)

[WinError 183] Cannot create a file when that file already exists: 'data/raw'


## Get DIA History

In [222]:
def max_start_and_end_data_for(symbol):
    df = yf.Ticker(symbol).history(period="max").iloc[[0, -1]]
    return add_symbol_as_top_index(df, symbol)

def add_symbol_as_top_index(df, symbol):
    return pd.concat({f'{symbol}': df}, names=['Symbol'])

This retrieves the start and end data for DIA and saves it locally.

Set `force` to `True` to run.

In [186]:
force = False

if force:
    dia_history = max_start_and_end_data_for("DIA")
    dia_history.to_pickle("data/raw/DIA.pickle")

Loads DIA history

In [187]:
dia_history = pd.read_pickle("data/raw/DIA.pickle")
dia_history

Open        High         Low       Close   Volume  \
Symbol Date                                                                  
DIA    1998-01-20   47.228395   47.854313   46.981821   47.835346  1744600   
       2021-01-14  310.630005  312.268311  310.130005  310.720001  1787230   

                   Dividends  Stock Splits  
Symbol Date                                 
DIA    1998-01-20        0.0             0  
       2021-01-14        0.0             0

## Get History for Dow Companies

In [248]:
def dow_symbols(path):
    with open(f'{path}.json', 'r') as fp:
        symbols = json.load(fp)
    return [list(k.keys())[0] for k in symbols]

def make_dummy_df(timestamp):
    df = pd.read_pickle("data/raw/DIA.pickle")
    df = (df
          .drop(index=df.index[0][1], level=1)  #drop 1st date row
          .droplevel(0))
    df = df.rename(index={df.index[0] : timestamp})
    df[df.columns] = np.nan
    return df

def start_and_end_data_for(symbol, start_timestamp, end_timestamp):
    start_end_timestamp = start_timestamp + dt.timedelta(days=1)
    end_end_timestamp = end_timestamp + dt.timedelta(days=1)
    
    start = yf.Ticker(symbol).history(start=start_timestamp, end=start_end_timestamp)
    end = yf.Ticker(symbol).history(start=end_timestamp, end=end_end_timestamp)
    
    if start.empty:
        start = make_dummy_df(start_timestamp)
    if end.empty:
        end = make_dummy_df(end_timestamp)
    
    return(pd.concat([start, end]))

Here we get the individual stocks in the Dow Jones based according to `data/dow-symbols` and the time period established by our benchmark, DIA and pickle them.

Set `force` to `True` to run.

In [249]:
force = False

if force:
    dow_sym = dow_symbols("data/dow-symbols")
    start_timestamp = dia_history.index[0][1]
    end_timestamp = dia_history.index[1][1]

    for s in dow_sym:
        df = start_and_end_data_for(s, start_timestamp, end_timestamp)
        combined = add_symbol_as_top_index(df, s)
        combined.to_pickle(f"data/raw/{s}.pickle")
        sleep(0.6)

Loads the dataframes from file and combines everything into a single `data/dow.csv`

In [252]:
def combine_pickles():
    pickles = [f for f in os.listdir("data/raw")]
    dataframes = [pd.read_pickle(f"data/raw/{p}") for p in pickles]
    return pd.concat(dataframes)

In [253]:
combine_pickles().to_csv("data/dow.csv")

Elsewhere, we can load our data from csv like so:

In [257]:
pd.read_csv("data/dow.csv", index_col=[0,1])

Open        High         Low       Close      Volume  \
Symbol Date                                                                     
AA     1998-01-20   28.716094   29.547440   28.681454   29.530121   2176900.0   
       2021-01-14   25.000000   25.379999   24.850000   25.090000   3988400.0   
AXP    1998-01-20   17.604445   17.765462   17.537355   17.698372   4635200.0   
       2021-01-14  122.650002  124.500000  122.089996  123.779999   3404200.0   
BA     1998-01-20   27.111473   27.618594   26.721380   27.150482   7730300.0   
...                       ...         ...         ...         ...         ...   
UK     2021-01-14         NaN         NaN         NaN         NaN         NaN   
WMT    1998-01-20   13.316331   14.115731   13.316331   14.052621   7682800.0   
       2021-01-14  147.820007  148.000000  146.039993  146.970001   7654400.0   
XOM    1998-01-20   15.468195   15.611567   15.372614   15.515985   6348000.0   
       2021-01-14   48.500000   51.080002   48.500000   50.310001  35429400.0   

                   Dividends  Stock Splits  
Symbol Date                                 
AA     1998-01-20        0.0           0.0  
       2021-01-14        0.0           0.0  
AXP    1998-01-20        0.0           0.0  
       2021-01-14        0.0           0.0  
BA     1998-01-20        0.0           0.0  
...                      ...           ...  
UK     2021-01-14        NaN           NaN  
WMT    1998-01-20        0.0           0.0  
       2021-01-14        0.0           0.0  
XOM    1998-01-20        0.0           0.0  
       2021-01-14        0.0           0.0  

[63 rows x 7 columns]